In [1]:
"""
SaaS Revenue Intelligence Analytics
Data Engineering and Business Analysis Pipeline
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("="*70)
print("Environment Configuration Complete")
print("="*70)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"Pandas Version: {pd.__version__}")
print("="*70)


Environment Configuration Complete
Analysis Date: 2025-10-23 20:03
Pandas Version: 2.1.1


In [2]:
# Load UCI Online Retail dataset
df_raw = pd.read_csv('Online_Retail.csv', encoding='ISO-8859-1')

print("="*70)
print("Raw Dataset Loaded")
print("="*70)
print(f"Total Rows: {df_raw.shape[0]:,}")
print(f"Total Columns: {df_raw.shape[1]}")
print(f"Date Range: {pd.to_datetime(df_raw['InvoiceDate']).min()} to {pd.to_datetime(df_raw['InvoiceDate']).max()}")
print(f"Memory Usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("="*70)

# Display first few rows
print("\nFirst 5 Rows:")
df_raw.head()


Raw Dataset Loaded
Total Rows: 541,909
Total Columns: 8
Date Range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Memory Usage: 195.86 MB

First 5 Rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,"17,850.00",United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,"17,850.00",United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,"17,850.00",United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,"17,850.00",United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,"17,850.00",United Kingdom


In [3]:
# Dataset structure and quality assessment
print("="*70)
print("Dataset Structure")
print("="*70)
df_raw.info()

print("\n" + "="*70)
print("Statistical Summary")
print("="*70)
print(df_raw.describe())

print("\n" + "="*70)
print("Missing Values Analysis")
print("="*70)
missing = pd.DataFrame({
    'Column': df_raw.columns,
    'Missing_Count': df_raw.isnull().sum(),
    'Missing_Percentage': (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
}).sort_values('Missing_Count', ascending=False)

print(missing)


Dataset Structure
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB

Statistical Summary
        Quantity  UnitPrice  CustomerID
count 541,909.00 541,909.00  406,829.00
mean        9.55       4.61   15,287.69
std       218.08      96.76    1,713.60
min   -80,995.00 -11,062.06   12,346.00
25%         1.00       1.25   13,953.00
50%         3.00       2.08   15,152.00
75%        10.00       4.13   16,791.00
max    80,995.00  38,970.00   18,287.0

In [4]:
# Identify data quality issues
print("="*70)
print("Data Quality Issues Detected")
print("="*70)

# Missing CustomerID
missing_customers = df_raw['CustomerID'].isnull().sum()
missing_pct = (missing_customers / len(df_raw)) * 100
print(f"Missing CustomerID: {missing_customers:,} rows ({missing_pct:.2f}%)")

# Negative Quantities (Returns/Cancellations)
negative_qty = df_raw[df_raw['Quantity'] < 0]
print(f"Negative Quantities: {len(negative_qty):,} rows ({len(negative_qty)/len(df_raw)*100:.2f}%)")

# Zero or Negative Prices
invalid_price = df_raw[df_raw['UnitPrice'] <= 0]
print(f"Invalid Prices (<=0): {len(invalid_price):,} rows ({len(invalid_price)/len(df_raw)*100:.2f}%)")

# Duplicates
duplicates = df_raw.duplicated().sum()
print(f"Duplicate Rows: {duplicates:,} ({duplicates/len(df_raw)*100:.2f}%)")

print("\n" + "="*70)
print("Sample of Problematic Data (Negative Quantities):")
print("="*70)
print(negative_qty.head(3))

# Store original count
original_count = len(df_raw)


Data Quality Issues Detected
Missing CustomerID: 135,080 rows (24.93%)
Negative Quantities: 10,624 rows (1.96%)
Invalid Prices (<=0): 2,517 rows (0.46%)
Duplicate Rows: 5,268 (0.97%)

Sample of Problematic Data (Negative Quantities):
    InvoiceNo StockCode                      Description  Quantity  \
141   C536379         D                         Discount        -1   
154   C536383    35004C  SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556   PLASTERS IN TIN CIRCUS PARADE        -12   

             InvoiceDate  UnitPrice  CustomerID         Country  
141  2010-12-01 09:41:00      27.50   14,527.00  United Kingdom  
154  2010-12-01 09:49:00       4.65   15,311.00  United Kingdom  
235  2010-12-01 10:24:00       1.65   17,548.00  United Kingdom  


In [5]:
# Data cleaning
print("="*70)
print("Starting Data Cleaning Process")
print("="*70)

# Create a copy for cleaning
df_clean = df_raw.copy()

# Step 1: Remove missing CustomerIDs
before = len(df_clean)
df_clean = df_clean[df_clean['CustomerID'].notna()]
after = len(df_clean)
print(f"Removed missing CustomerIDs: {before - after:,} rows")

# Step 2: Remove negative quantities
before = len(df_clean)
df_clean = df_clean[df_clean['Quantity'] > 0]
after = len(df_clean)
print(f"Removed negative quantities: {before - after:,} rows")

# Step 3: Remove invalid prices
before = len(df_clean)
df_clean = df_clean[df_clean['UnitPrice'] > 0]
after = len(df_clean)
print(f"Removed invalid prices: {before - after:,} rows")

# Step 4: Remove duplicates
before = len(df_clean)
df_clean = df_clean.drop_duplicates()
after = len(df_clean)
print(f"Removed duplicates: {before - after:,} rows")

# Calculate retention rate
retention_rate = (len(df_clean) / original_count) * 100

print("\n" + "="*70)
print("Cleaning Summary")
print("="*70)
print(f"Original Dataset: {original_count:,} rows")
print(f"Cleaned Dataset: {len(df_clean):,} rows")
print(f"Rows Removed: {original_count - len(df_clean):,}")
print(f"Retention Rate: {retention_rate:.1f}%")
print("="*70)

# Convert date column
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Create Revenue column
df_clean['Revenue'] = df_clean['Quantity'] * df_clean['UnitPrice']

print("\nData cleaning completed successfully")
print(f"Clean dataset shape: {df_clean.shape}")


Starting Data Cleaning Process
Removed missing CustomerIDs: 135,080 rows
Removed negative quantities: 8,905 rows
Removed invalid prices: 40 rows
Removed duplicates: 5,192 rows

Cleaning Summary
Original Dataset: 541,909 rows
Cleaned Dataset: 392,692 rows
Rows Removed: 149,217
Retention Rate: 72.5%

Data cleaning completed successfully
Clean dataset shape: (392692, 9)


In [6]:
# Transform transactions to SaaS subscription model
print("="*70)
print("SaaS Transformation: Aggregating to Monthly Subscriptions")
print("="*70)

# Extract year-month from invoice date
df_clean['YearMonth'] = df_clean['InvoiceDate'].dt.to_period('M')

# Aggregate to monthly customer level (MRR - Monthly Recurring Revenue)
mrr_data = df_clean.groupby(['CustomerID', 'YearMonth']).agg({
    'Revenue': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'count'
}).reset_index()

# Rename columns
mrr_data.columns = ['CustomerID', 'Month', 'MRR', 'Items_Purchased', 'Transaction_Count']

# Convert period back to timestamp for easier handling
mrr_data['Month'] = mrr_data['Month'].dt.to_timestamp()

print(f"Transaction-level records: {len(df_clean):,}")
print(f"Monthly subscription records: {len(mrr_data):,}")
print(f"Unique customers: {mrr_data['CustomerID'].nunique():,}")
print(f"Date range: {mrr_data['Month'].min()} to {mrr_data['Month'].max()}")

print("\nSample of MRR data:")
print(mrr_data.head(10))


SaaS Transformation: Aggregating to Monthly Subscriptions
Transaction-level records: 392,692
Monthly subscription records: 13,054
Unique customers: 4,338
Date range: 2010-12-01 00:00:00 to 2011-12-01 00:00:00

Sample of MRR data:
   CustomerID      Month       MRR  Items_Purchased  Transaction_Count
0   12,346.00 2011-01-01 77,183.60            74215                  1
1   12,347.00 2010-12-01    711.79              319                 31
2   12,347.00 2011-01-01    475.39              315                 29
3   12,347.00 2011-04-01    636.25              483                 24
4   12,347.00 2011-06-01    382.52              196                 18
5   12,347.00 2011-08-01    584.91              277                 22
6   12,347.00 2011-10-01  1,294.32              676                 47
7   12,347.00 2011-12-01    224.82              192                 11
8   12,348.00 2010-12-01    892.80             1254                 17
9   12,348.00 2011-01-01    227.44              601         

In [7]:
# Calculate customer-level aggregated metrics
print("="*70)
print("Calculating Customer Lifetime Metrics")
print("="*70)

customer_summary = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': ['min', 'max'],
    'InvoiceNo': 'nunique',
    'Revenue': 'sum',
    'Quantity': 'sum'
}).reset_index()

# Flatten column names
customer_summary.columns = ['CustomerID', 'First_Purchase', 'Last_Purchase', 
                            'Total_Orders', 'Total_Revenue', 'Total_Items']

# Calculate customer lifetime in days
customer_summary['Lifetime_Days'] = (customer_summary['Last_Purchase'] - 
                                      customer_summary['First_Purchase']).dt.days

# Calculate average order value
customer_summary['Avg_Order_Value'] = customer_summary['Total_Revenue'] / customer_summary['Total_Orders']

print(f"Total customers analyzed: {len(customer_summary):,}")
print("\nCustomer Lifetime Metrics:")
print(customer_summary.describe())

print("\nTop 10 Customers by Revenue:")
print(customer_summary.nlargest(10, 'Total_Revenue')[['CustomerID', 'Total_Revenue', 'Total_Orders']])


Calculating Customer Lifetime Metrics
Total customers analyzed: 4,338

Customer Lifetime Metrics:
       CustomerID                 First_Purchase  \
count    4,338.00                           4338   
mean    15,300.41  2011-04-30 17:06:50.857538048   
min     12,346.00            2010-12-01 08:26:00   
25%     13,813.25            2011-01-17 11:13:15   
50%     15,299.50            2011-04-05 09:52:30   
75%     16,778.75            2011-08-19 10:11:30   
max     18,287.00            2011-12-09 12:16:00   
std      1,721.81                            NaN   

                       Last_Purchase  Total_Orders  Total_Revenue  \
count                           4338      4,338.00       4,338.00   
mean   2011-09-08 11:38:59.045643008          4.27       2,048.69   
min              2010-12-01 09:53:00          1.00           3.75   
25%              2011-07-20 19:18:00          1.00         306.48   
50%              2011-10-20 10:40:30          2.00         668.57   
75%              20

In [8]:
# RFM Segmentation
print("="*70)
print("RFM Analysis: Customer Segmentation")
print("="*70)

# Define analysis date (day after last transaction)
analysis_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

# Calculate RFM metrics
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (analysis_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'Revenue': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Create RFM scores (1-5 scale, higher is better)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Combine scores
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Total'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

print("RFM Metrics Summary:")
print(rfm[['Recency', 'Frequency', 'Monetary']].describe())

print("\nSample RFM Scores:")
print(rfm.head(10))


RFM Analysis: Customer Segmentation
RFM Metrics Summary:
       Recency  Frequency   Monetary
count 4,338.00   4,338.00   4,338.00
mean     92.54       4.27   2,048.69
std     100.01       7.70   8,985.23
min       1.00       1.00       3.75
25%      18.00       1.00     306.48
50%      51.00       2.00     668.57
75%     142.00       5.00   1,660.60
max     374.00     209.00 280,206.02

Sample RFM Scores:
   CustomerID  Recency  Frequency  Monetary R_Score F_Score M_Score RFM_Score  \
0   12,346.00      326          1 77,183.60       1       1       5       115   
1   12,347.00        2          7  4,310.00       5       5       5       555   
2   12,348.00       75          4  1,797.24       2       4       4       244   
3   12,349.00       19          1  1,757.55       4       1       4       414   
4   12,350.00      310          1    334.40       1       1       2       112   
5   12,352.00       36          8  2,506.04       3       5       5       355   
6   12,353.00      204 

In [9]:
# Segment customers based on RFM scores
print("="*70)
print("Customer Segmentation Based on RFM")
print("="*70)

def segment_customers(row):
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3 and m >= 3:
        return 'Loyal Customers'
    elif r >= 4 and f <= 2 and m <= 2:
        return 'New Customers'
    elif r <= 2 and f >= 4 and m >= 4:
        return 'At Risk'
    elif r <= 2 and f <= 2:
        return 'Lost'
    elif r >= 3 and f <= 2 and m <= 2:
        return 'Promising'
    elif r <= 2 and f >= 3:
        return 'Need Attention'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(segment_customers, axis=1)

# Segment distribution
segment_counts = rfm['Segment'].value_counts()
print("\nCustomer Segment Distribution:")
print(segment_counts)

# Calculate segment value
segment_value = rfm.groupby('Segment').agg({
    'CustomerID': 'count',
    'Monetary': ['sum', 'mean'],
    'Frequency': 'mean',
    'Recency': 'mean'
}).round(2)

print("\nSegment Performance Metrics:")
print(segment_value)


Customer Segmentation Based on RFM

Customer Segment Distribution:
Segment
Lost               1065
Champions           957
Loyal Customers     764
Need Attention      475
Others              424
Promising           250
New Customers       235
At Risk             168
Name: count, dtype: int64

Segment Performance Metrics:
                CustomerID     Monetary          Frequency Recency
                     count          sum     mean      mean    mean
Segment                                                           
At Risk                168   370,498.85 2,205.35      5.53  125.56
Champions              957 5,791,640.74 6,051.87     11.12   12.82
Lost                  1065   518,322.15   486.69      1.10  217.90
Loyal Customers        764 1,396,513.16 1,827.90      4.12   35.69
Need Attention         475   427,553.66   900.11      2.65  162.49
New Customers          235    56,697.16   241.26      1.12   18.77
Others                 424   262,795.78   619.80      2.01   32.56
Promisi

In [10]:
# Calculate churn risk based on recency and declining purchase patterns
print("="*70)
print("Churn Risk Analysis")
print("="*70)

# Merge rfm with customer summary
customer_analysis = customer_summary.merge(rfm[['CustomerID', 'Recency', 'Segment', 'RFM_Total']], 
                                           on='CustomerID')

# Calculate churn risk score (0-100, higher = more risk)
def calculate_churn_risk(row):
    risk_score = 0
    
    # Recency factor (40% weight)
    if row['Recency'] > 180:
        risk_score += 40
    elif row['Recency'] > 90:
        risk_score += 30
    elif row['Recency'] > 60:
        risk_score += 20
    elif row['Recency'] > 30:
        risk_score += 10
    
    # Frequency factor (30% weight)
    if row['Total_Orders'] < 5:
        risk_score += 30
    elif row['Total_Orders'] < 10:
        risk_score += 20
    elif row['Total_Orders'] < 20:
        risk_score += 10
    
    # RFM score factor (30% weight)
    if row['RFM_Total'] < 6:
        risk_score += 30
    elif row['RFM_Total'] < 9:
        risk_score += 20
    elif row['RFM_Total'] < 12:
        risk_score += 10
    
    return risk_score

customer_analysis['Churn_Risk_Score'] = customer_analysis.apply(calculate_churn_risk, axis=1)

# Categorize churn risk
def churn_risk_category(score):
    if score >= 70:
        return 'High Risk'
    elif score >= 40:
        return 'Medium Risk'
    else:
        return 'Low Risk'

customer_analysis['Churn_Risk_Category'] = customer_analysis['Churn_Risk_Score'].apply(churn_risk_category)

print("Churn Risk Distribution:")
print(customer_analysis['Churn_Risk_Category'].value_counts())

print("\nHigh Risk Customers:")
high_risk = customer_analysis[customer_analysis['Churn_Risk_Category'] == 'High Risk']
print(f"Count: {len(high_risk):,}")
print(f"Total Revenue at Risk: ${high_risk['Total_Revenue'].sum():,.2f}")


Churn Risk Analysis
Churn Risk Distribution:
Churn_Risk_Category
High Risk      1731
Medium Risk    1425
Low Risk       1182
Name: count, dtype: int64

High Risk Customers:
Count: 1,731
Total Revenue at Risk: $1,046,682.67


In [11]:
"""
MRR GROWTH ACCOUNTING ANALYSIS

Purpose: Track how Monthly Recurring Revenue changes month-over-month
Categories: New, Expansion, Contraction, Reactivation, Retained

Key Concept: Compare each month's MRR to the previous month for the same customer
- New: First time seeing this customer (no previous month MRR)
- Expansion: Customer's MRR increased (they spent more)
- Contraction: Customer's MRR decreased (they spent less)
- Reactivation: Customer was gone for 2+ months, now back
- Retained: Customer's MRR stayed the same
"""

print("="*70)
print("MRR Growth Accounting Analysis")
print("="*70)

# Step 1: Sort data by customer and month (chronological order per customer)
# Why: We need to compare each month to the PREVIOUS month for the SAME customer
mrr_sorted = mrr_data.sort_values(['CustomerID', 'Month']).reset_index(drop=True)

# Step 2: Get previous month's data for each customer
# .shift(1) moves each row down by 1, so we can see "last month" next to "this month"
mrr_sorted['Prev_Month'] = mrr_sorted.groupby('CustomerID')['Month'].shift(1)
mrr_sorted['Prev_MRR'] = mrr_sorted.groupby('CustomerID')['MRR'].shift(1)

# Step 3: Calculate how many months passed since last transaction
# Why: If gap > 1 month, customer "churned" then came back (Reactivation)
# Convert days to months by dividing by 30
mrr_sorted['Months_Since_Last'] = ((mrr_sorted['Month'] - mrr_sorted['Prev_Month']).dt.days / 30).round()

# Step 4: Categorize each transaction based on MRR change
def categorize_mrr_change(row):
    # No previous MRR? This is their first month (New customer)
    if pd.isna(row['Prev_MRR']):
        return 'New'
    
    # Gap of 2+ months? They were gone, now back (Reactivation)
    elif row['Months_Since_Last'] > 1:
        return 'Reactivation'
    
    # MRR went up? Expansion (upgrade/increased usage)
    elif row['MRR'] > row['Prev_MRR']:
        return 'Expansion'
    
    # MRR went down? Contraction (downgrade/decreased usage)
    elif row['MRR'] < row['Prev_MRR']:
        return 'Contraction'
    
    # MRR stayed same? Retained (stable customer)
    else:
        return 'Retained'

# Apply the categorization to each row
mrr_sorted['MRR_Type'] = mrr_sorted.apply(categorize_mrr_change, axis=1)

# Step 5: Calculate the dollar amount of MRR change
# If Prev_MRR is NaN (new customer), treat it as 0
mrr_sorted['MRR_Change'] = mrr_sorted['MRR'] - mrr_sorted['Prev_MRR'].fillna(0)

# Display results
print("\nMRR Type Distribution:")
print(mrr_sorted['MRR_Type'].value_counts())

print("\nMRR Change Breakdown by Type:")
mrr_summary = mrr_sorted.groupby('MRR_Type').agg({
    'CustomerID': 'count',      # How many transactions in each category
    'MRR_Change': 'sum',         # Total dollar change
    'MRR': 'sum'                 # Total MRR
}).round(2)
mrr_summary.columns = ['Count', 'Total_MRR_Change', 'Total_MRR']
print(mrr_summary)

# Step 6: Monthly MRR movement (see trends over time)
monthly_mrr_movement = mrr_sorted.groupby(['Month', 'MRR_Type']).agg({
    'MRR_Change': 'sum',
    'CustomerID': 'count'
}).reset_index()

print("\nSample Monthly MRR Movement:")
print(monthly_mrr_movement.head(15))

# Save for later use
mrr_with_changes = mrr_sorted.copy()


MRR Growth Accounting Analysis

MRR Type Distribution:
MRR_Type
New             4338
Reactivation    4098
Contraction     2449
Expansion       2158
Retained          11
Name: count, dtype: int64

MRR Change Breakdown by Type:
              Count  Total_MRR_Change    Total_MRR
MRR_Type                                          
Contraction    2449     -1,640,512.14 1,518,395.64
Expansion      2158      1,460,429.15 2,830,495.96
New            4338      2,244,830.59 2,244,830.59
Reactivation   4098        260,854.65 2,288,099.20
Retained         11              0.00     5,387.50

Sample Monthly MRR Movement:
        Month      MRR_Type  MRR_Change  CustomerID
0  2010-12-01           New  570,422.73         885
1  2011-01-01   Contraction -124,374.96         180
2  2011-01-01     Expansion  100,049.91         144
3  2011-01-01           New  292,366.84         417
4  2011-02-01   Contraction  -58,091.93         136
5  2011-02-01     Expansion   51,292.26         125
6  2011-02-01          

In [13]:
"""
=============================================================================
SYNTHETIC DATA GENERATION: SUPPORT TICKETS
=============================================================================

Purpose: Generate realistic customer support ticket data that matches
         the existing PostgreSQL table structure used in Power BI dashboards

Table Structure (PostgreSQL): supporttickets
Columns: TicketID, CustomerID, CreatedTimestamp, TicketType, Sentiment, Segment

Correlation Strategy:
- Customers with declining revenue trends → More tickets + Negative sentiment
- High churn risk customers → More support interactions
- Different RFM segments → Different ticket patterns and sentiments

This demonstrates:
1. Consistent data schema across pipeline (Python → CSV → PostgreSQL → Power BI)
2. Business logic correlation (support behavior linked to customer health)
3. Synthetic data generation with realistic distributions
"""

print("="*70)
print("Generating Synthetic Support Tickets Data")
print("="*70)

# Step 1: Get customer data with segments and revenue trends
# Merge customer_analysis with revenue trend calculation
customer_revenue_trends = df_clean.groupby('CustomerID').apply(
    lambda x: (x.sort_values('InvoiceDate')['Revenue'].iloc[-1] - 
               x.sort_values('InvoiceDate')['Revenue'].iloc[0]) 
              if len(x) > 1 else 0
).reset_index()
customer_revenue_trends.columns = ['CustomerID', 'Revenue_Trend']

support_base = customer_analysis[['CustomerID', 'Segment']].merge(
    customer_revenue_trends, on='CustomerID', how='left'
)

# Step 2: Define ticket generation rules based on revenue trends
# Declining revenue = more tickets (struggling customers need help)
def generate_ticket_count(revenue_trend):
    """
    Determine number of tickets based on customer revenue trend
    Declining revenue = more support issues
    Growing revenue = fewer support issues
    """
    if revenue_trend < 0:      # Revenue declining
        return np.random.randint(3, 10)
    elif revenue_trend == 0:    # Stable
        return np.random.randint(1, 4)
    else:                       # Revenue growing
        return np.random.randint(0, 3)

# Apply ticket count generation
support_base['Ticket_Count'] = support_base['Revenue_Trend'].fillna(0).apply(generate_ticket_count)

# Step 3: Create individual ticket records
ticket_records = []

# Ticket types realistic to SaaS businesses (matching your original)
ticket_types = ['Technical Issue', 'Billing Question', 'Feature Request', 
                'Account Access', 'Product Training']

# Sentiment categories (matching PostgreSQL table)
sentiments = ['Positive', 'Neutral', 'Negative']

# Set random seed for reproducibility
np.random.seed(42)

# Step 4: Generate tickets for each customer
ticket_id_counter = 1

for idx, row in support_base.iterrows():
    customer_id = row['CustomerID']
    ticket_count = row['Ticket_Count']
    revenue_trend = row['Revenue_Trend']
    segment = row['Segment']
    
    # Generate multiple tickets per customer
    for i in range(ticket_count):
        # Random ticket date within customer's active period
        customer_dates = df_clean[df_clean['CustomerID'] == customer_id]['InvoiceDate']
        if len(customer_dates) > 0:
            min_date = customer_dates.min()
            max_date = customer_dates.max()
            days_range = (max_date - min_date).days
            if days_range > 0:
                ticket_date = min_date + pd.Timedelta(days=np.random.randint(0, days_range))
            else:
                ticket_date = min_date
        else:
            ticket_date = pd.Timestamp('2011-06-01')
        
        # Ticket type weighted by revenue trend and segment
        if revenue_trend < 0:  # Declining revenue
            # More likely to have technical issues and billing questions
            ticket_type = np.random.choice(
                ['Technical Issue', 'Billing Question', 'Account Access'],
                p=[0.5, 0.3, 0.2]
            )
            # Negative sentiment for struggling customers
            sentiment = np.random.choice(sentiments, p=[0.1, 0.3, 0.6])
        elif segment in ['Champions', 'Loyal']:
            # High-value customers: feature requests, positive sentiment
            ticket_type = np.random.choice(['Feature Request', 'Product Training'])
            sentiment = np.random.choice(sentiments, p=[0.6, 0.3, 0.1])
        else:
            # Random distribution for others
            ticket_type = np.random.choice(ticket_types)
            sentiment = np.random.choice(sentiments, p=[0.3, 0.4, 0.3])
        
        # Create ticket record (EXACT column names matching PostgreSQL)
        ticket_records.append({
            'TicketID': ticket_id_counter,
            'CustomerID': customer_id,
            'CreatedTimestamp': ticket_date,
            'TicketType': ticket_type,
            'Sentiment': sentiment,
            'Segment': segment
        })
        
        ticket_id_counter += 1

# Step 5: Create tickets dataframe
support_tickets = pd.DataFrame(ticket_records)

# Display summary
print(f"\nTotal Support Tickets Generated: {len(support_tickets):,}")
print(f"Customers with Tickets: {support_tickets['CustomerID'].nunique():,}")

print("\nTickets by Segment:")
print(support_tickets['Segment'].value_counts())

print("\nTicket Type Distribution:")
print(support_tickets['TicketType'].value_counts())

print("\nSentiment Distribution:")
print(support_tickets['Sentiment'].value_counts())

print("\nSample Tickets:")
print(support_tickets.head(10))


Generating Synthetic Support Tickets Data

Total Support Tickets Generated: 13,854
Customers with Tickets: 3,566

Tickets by Segment:
Segment
Lost               3452
Champions          3053
Loyal Customers    2495
Need Attention     1466
Others             1388
New Customers       771
Promising           713
At Risk             516
Name: count, dtype: int64

Ticket Type Distribution:
TicketType
Technical Issue     6006
Billing Question    3830
Account Access      2627
Product Training     709
Feature Request      682
Name: count, dtype: int64

Sentiment Distribution:
Sentiment
Negative    7373
Neutral     4433
Positive    2048
Name: count, dtype: int64

Sample Tickets:
   TicketID  CustomerID    CreatedTimestamp        TicketType Sentiment  \
0         1   12,346.00 2011-01-18 10:01:00    Account Access  Negative   
1         2   12,347.00 2011-03-23 14:57:00  Billing Question  Negative   
2         3   12,347.00 2011-04-07 14:57:00   Technical Issue  Positive   
3         4   12,347.0

In [14]:
"""
=============================================================================
SYNTHETIC DATA GENERATION: USAGE LOGS
=============================================================================

Purpose: Generate realistic product usage/activity data that matches
         the existing PostgreSQL table structure used in Power BI dashboards

Table Structure (PostgreSQL): usagelogs
Columns: LogID, CustomerID, EventTimestamp, EventType, SessionDuration

Correlation Strategy:
- Purchase frequency → Login frequency (active buyers = active users)
- RFM segment → Usage patterns:
  * Champions/Loyal: High login frequency, longer sessions
  * At Risk/Lost: Low login frequency, shorter sessions
  * New: Moderate frequency, learning behavior

This demonstrates:
1. Behavioral correlation (engagement drives revenue)
2. Realistic usage patterns by customer lifecycle stage
3. Time-series event generation
"""

print("="*70)
print("Generating Synthetic Usage Logs Data")
print("="*70)

# Step 1: Calculate purchase frequency for each customer
# More purchases = more engaged = more logins
purchase_frequency = df_clean.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',
    'InvoiceDate': ['min', 'max']
}).reset_index()
purchase_frequency.columns = ['CustomerID', 'Purchase_Count', 'First_Purchase', 'Last_Purchase']

# Merge with segment data
usage_base = customer_analysis[['CustomerID', 'Segment']].merge(
    purchase_frequency, on='CustomerID', how='left'
)

# Step 2: Define login generation rules based on purchase frequency
# Active buyers = active users (correlated behavior)
def generate_login_count(purchase_count):
    """
    Determine number of logins based on purchase frequency
    More purchases = more platform engagement
    Use Poisson distribution for realistic variance
    """
    avg_logins_per_month = max(1, int(purchase_count / 2))
    return np.random.poisson(avg_logins_per_month)

# Apply login count generation
usage_base['Login_Count'] = usage_base['Purchase_Count'].fillna(1).apply(generate_login_count)

# Step 3: Create individual usage log records
usage_records = []

# Event types realistic to SaaS platform usage
event_types = ['Login', 'Dashboard View', 'Report Generated', 
               'Data Export', 'Settings Updated', 'Profile View']

# Set random seed for reproducibility
np.random.seed(42)

# Step 4: Generate usage logs for each customer
log_id_counter = 1

for idx, row in usage_base.iterrows():
    customer_id = row['CustomerID']
    login_count = row['Login_Count']
    segment = row['Segment']
    first_date = row['First_Purchase']
    last_date = row['Last_Purchase']
    
    # Generate multiple login events per customer
    for i in range(login_count):
        # Random event date within customer's active period
        if pd.notna(first_date) and pd.notna(last_date):
            days_range = (last_date - first_date).days
            if days_range > 0:
                event_date = first_date + pd.Timedelta(days=np.random.randint(0, days_range))
            else:
                event_date = first_date
        else:
            event_date = pd.Timestamp('2011-06-01')
        
        # Event type weighted by segment
        # Champions/Loyal use advanced features more
        if segment in ['Champions', 'Loyal Customers']:
            event_type = np.random.choice(event_types, p=[0.3, 0.2, 0.2, 0.15, 0.1, 0.05])
            # Longer sessions for engaged users
            session_duration = np.random.randint(15, 60)
        elif segment in ['At Risk', 'Lost']:
            # At-risk users mostly just login, short sessions
            event_type = np.random.choice(['Login', 'Dashboard View'], p=[0.7, 0.3])
            session_duration = np.random.randint(2, 15)
        else:
            # Average usage for others
            event_type = np.random.choice(event_types)
            session_duration = np.random.randint(5, 30)
        
        # Create usage log record (EXACT column names matching PostgreSQL)
        usage_records.append({
            'LogID': log_id_counter,
            'CustomerID': customer_id,
            'EventTimestamp': event_date,
            'EventType': event_type,
            'SessionDuration': session_duration
        })
        
        log_id_counter += 1

# Step 5: Create usage logs dataframe
usage_logs = pd.DataFrame(usage_records)

# Display summary
print(f"\nTotal Usage Logs Generated: {len(usage_logs):,}")
print(f"Customers with Activity: {usage_logs['CustomerID'].nunique():,}")

print("\nUsage by Segment:")
usage_by_segment = usage_logs.merge(
    customer_analysis[['CustomerID', 'Segment']], 
    on='CustomerID'
)
print(usage_by_segment['Segment'].value_counts())

print("\nEvent Type Distribution:")
print(usage_logs['EventType'].value_counts())

print("\nSession Duration Stats:")
print(usage_logs['SessionDuration'].describe())

print("\nSample Usage Logs:")
print(usage_logs.head(10))


Generating Synthetic Usage Logs Data

Total Usage Logs Generated: 9,481
Customers with Activity: 3,241

Usage by Segment:
Segment
Champions          5040
Loyal Customers    1426
Lost               1092
Need Attention      529
At Risk             448
Others              442
Promising           259
New Customers       245
Name: count, dtype: int64

Event Type Distribution:
EventType
Login               3279
Dashboard View      1982
Report Generated    1500
Data Export         1220
Settings Updated     920
Profile View         580
Name: count, dtype: int64

Session Duration Stats:
count   9,481.00
mean       29.12
std        16.23
min         2.00
25%        16.00
50%        27.00
75%        43.00
max        59.00
Name: SessionDuration, dtype: float64

Sample Usage Logs:
   LogID  CustomerID      EventTimestamp       EventType  SessionDuration
0      1   12,347.00 2011-03-19 14:57:00     Data Export               29
1      2   12,347.00 2011-03-23 14:57:00     Data Export               35

In [15]:
"""
=============================================================================
DATA EXPORT: Save All Processed Datasets to CSV
=============================================================================

Purpose: Export all cleaned and generated datasets for documentation
         This demonstrates the complete data pipeline from raw to processed

File Naming Convention: saas_*.csv
- Matches existing production file naming
- Shows complete transformation process
- Same schema as PostgreSQL tables for consistency

Output Files:
1. saas_cleaned_transactions.csv - Cleaned retail data
2. saas_mrr_subscriptions.csv - Monthly subscription revenue
3. saas_customer_analysis.csv - RFM + Churn analysis
4. saas_support_tickets.csv - Synthetic support data
5. saas_usage_logs.csv - Synthetic usage data
"""

print("="*70)
print("Exporting All Datasets to CSV")
print("="*70)

# Export 1: Cleaned Transactions
print("\n1. Exporting Cleaned Transactions...")
df_clean.to_csv('saas_cleaned_transactions.csv', index=False)
print(f"   Saved: {len(df_clean):,} rows -> saas_cleaned_transactions.csv")

# Export 2: MRR Subscriptions
print("\n2. Exporting MRR Subscriptions...")
mrr_data.to_csv('saas_mrr_subscriptions.csv', index=False)
print(f"   Saved: {len(mrr_data):,} rows -> saas_mrr_subscriptions.csv")

# Export 3: Customer Analysis
print("\n3. Exporting Customer Analysis...")
customer_analysis.to_csv('saas_customer_analysis.csv', index=False)
print(f"   Saved: {len(customer_analysis):,} rows -> saas_customer_analysis.csv")

# Export 4: Support Tickets
print("\n4. Exporting Support Tickets...")
support_tickets.to_csv('saas_support_tickets.csv', index=False)
print(f"   Saved: {len(support_tickets):,} rows -> saas_support_tickets.csv")

# Export 5: Usage Logs
print("\n5. Exporting Usage Logs...")
usage_logs.to_csv('saas_usage_logs.csv', index=False)
print(f"   Saved: {len(usage_logs):,} rows -> saas_usage_logs.csv")

print("\n" + "="*70)
print("Export Complete!")
print("="*70)
print("\nAll files saved to current directory with 'saas_' prefix")
print("\nFiles created:")
print("  - saas_cleaned_transactions.csv")
print("  - saas_mrr_subscriptions.csv")
print("  - saas_customer_analysis.csv")
print("  - saas_support_tickets.csv")
print("  - saas_usage_logs.csv")

print("\nNote: Files match PostgreSQL table schemas for direct import")


Exporting All Datasets to CSV

1. Exporting Cleaned Transactions...
   Saved: 392,692 rows -> saas_cleaned_transactions.csv

2. Exporting MRR Subscriptions...
   Saved: 13,054 rows -> saas_mrr_subscriptions.csv

3. Exporting Customer Analysis...
   Saved: 4,338 rows -> saas_customer_analysis.csv

4. Exporting Support Tickets...
   Saved: 13,854 rows -> saas_support_tickets.csv

5. Exporting Usage Logs...
   Saved: 9,481 rows -> saas_usage_logs.csv

Export Complete!

All files saved to current directory with 'saas_' prefix

Files created:
  - saas_cleaned_transactions.csv
  - saas_mrr_subscriptions.csv
  - saas_customer_analysis.csv
  - saas_support_tickets.csv
  - saas_usage_logs.csv

Note: Files match PostgreSQL table schemas for direct import
